In [1]:
import warnings
warnings.simplefilter('ignore')

In [7]:
!pip install pystan -q
!pip install prophet -q

In [14]:
import pandas as pd
from prophet import Prophet

The input to Prophet is always a dataframe with two columns: `ds` and `y`. The `ds` (datestamp) column should be of a format expected by Pandas, ideally `YYYY-MM-DD` for a date or `YYYY-MM-DD HH:MM:SS` for a timestamp. The `y` column must be numeric, and represents the measurement we wish to forecast.

In [15]:
df = pd.read_csv('https://raw.githubusercontent.com/facebook/prophet/main/examples/example_wp_log_peyton_manning.csv')

In [21]:
df.tail()

,ds,y
2900,2016-01-16,7.817223
2901,2016-01-17,9.273878
2902,2016-01-18,10.333775
2903,2016-01-19,9.125871
2904,2016-01-20,8.891374


In [13]:
m = Prophet()
m.fit(df)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpchgfn8wf/xhll7m0c.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpchgfn8wf/5ces8gul.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=70766', 'data', 'file=/tmp/tmpchgfn8wf/xhll7m0c.json', 'init=/tmp/tmpchgfn8wf/5ces8gul.json', 'output', 'file=/tmp/tmpchgfn8wf/prophet_modelarmdcyj9/prophet_model-20230702092512.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
09:25:12 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
09:25:12 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [17]:
future = m.make_future_dataframe(periods=365)

In [18]:
future.tail()

,ds
3265,2017-01-15
3266,2017-01-16
3267,2017-01-17
3268,2017-01-18
3269,2017-01-19


The `predict` method will assign each row in future a predicted value which it names `yhat`. If you pass in historical dates, it will provide an in-sample fit. The forecast object here is a new dataframe that includes a column `yhat` with the forecast, as well as columns for components and uncertainty intervals.

In [19]:
forecast = m.predict(future)

In [26]:
forecast.tail()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
3265,2017-01-15,7.190267,7.435603,8.897860,6.847635,7.506439,1.018038,1.018038,1.018038,0.048296,0.048296,0.048296,0.969742,0.969742,0.969742,0.0,0.0,0.0,8.208305
3266,2017-01-16,7.189242,7.810413,9.286673,6.844271,7.506984,1.344067,1.344067,1.344067,0.352288,0.352288,0.352288,0.991779,0.991779,0.991779,0.0,0.0,0.0,8.533309
3267,2017-01-17,7.188218,7.674037,9.075612,6.842285,7.509778,1.132496,1.132496,1.132496,0.119622,0.119622,0.119622,1.012874,1.012874,1.012874,0.0,0.0,0.0,8.320714
3268,2017-01-18,7.187193,7.395200,8.860515,6.840461,7.510486,0.966163,0.966163,0.966163,-0.066659,-0.066659,-0.066659,1.032822,1.032822,1.032822,0.0,0.0,0.0,8.153356
3269,2017-01-19,7.186168,7.444548,8.898080,6.838637,7.511054,0.979110,0.979110,0.979110,-0.072268,-0.072268,-0.072268,1.051379,1.051379,1.051379,0.0,0.0,0.0,8.165279


In [20]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
3265,2017-01-15,8.208305,7.435603,8.897860
3266,2017-01-16,8.533309,7.810413,9.286673
3267,2017-01-17,8.320714,7.674037,9.075612
3268,2017-01-18,8.153356,7.395200,8.860515
3269,2017-01-19,8.165279,7.444548,8.898080


In [24]:
from prophet.plot import plot_plotly, plot_components_plotly

plot_plotly(m, forecast)

In [25]:
plot_components_plotly(m, forecast)